In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Loading the model
from langchain.chat_models import init_chat_model
model = init_chat_model("gemma2-9b-it", model_provider="groq")

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi My name is Anas and I am a AI Engineer.")
])

AIMessage(content="Hi Anas, it's nice to meet you! That's a fascinating field. \n\nWhat kind of AI engineering are you focused on?  Do you have any particular projects you're working on that you'd like to talk about?  I'm always interested in learning more about what people are doing in the world of AI. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 21, 'total_tokens': 98, 'completion_time': 0.14, 'prompt_time': 0.002118051, 'queue_time': 0.268114608, 'total_time': 0.142118051}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--06117cf3-ed95-4a6e-9390-5ef1ff321f7c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 77, 'total_tokens': 98})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Anas Malik and I am an AI Engineer."),
    AIMessage(content="Hi Anas, it's nice to meet you! That's a fascinating field. \n\nWhat kind of AI engineering are you focused on?"),
    HumanMessage(content="Hey, what's my name and what do I do ?")
])

AIMessage(content="You are Anas Malik, and you are an AI Engineer! \n\nIs there anything else you'd like to know or discuss? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 76, 'total_tokens': 108, 'completion_time': 0.058181818, 'prompt_time': 0.00380981, 'queue_time': 0.26055656, 'total_time': 0.061991628}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--dc91de78-4d9e-4069-8398-76ddab44f78c-0', usage_metadata={'input_tokens': 76, 'output_tokens': 32, 'total_tokens': 108})

## Message History

In [5]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [6]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [8]:
with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history
)

In [7]:
config = {"configurable": {"session_id": "chat1"}}

In [9]:
with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Anas Malik and I am an AI Engineer.")
    ],
    config=config
)

AIMessage(content="Hi Anas Malik, it's nice to meet you!\n\nBeing an AI Engineer is fascinating. What kind of projects are you working on these days?  \n\nI'm always eager to learn more about the work people are doing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 23, 'total_tokens': 79, 'completion_time': 0.101818182, 'prompt_time': 0.0026179, 'queue_time': 0.26785173, 'total_time': 0.104436082}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6522adc1-ab73-45ac-8128-1933878340d9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 56, 'total_tokens': 79})

In [10]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name and what do I do ?")
    ],
    config=config
)
response.content

"You said your name is Anas Malik and that you are an AI Engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat!\n"

In [11]:
# Change the config
config = {"configurable": {"session_id": "chat2"}}

In [12]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name and what do I do ?")
    ],
    config=config
)
response.content

'As a large language model, I have no memory of past conversations and do not know your name or what you do.\n\nPlease tell me, what is your name? 😊\n'

In [13]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Oh! My name is john. Nice to meet you.")
    ],
    config=config
)
response.content

'Nice to meet you too, John! \n\nWhat can I do for you today? 😊  \n'

In [14]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Now can you tell me my name ?")
    ],
    config=config
)
response.content

'Your name is John, right? 😊  I remember!  \n\n\nIs there anything else I can help you with?\n'

## Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all of the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="chat_history")
    ]
)

In [20]:
chain = prompt | model

In [21]:
chain.invoke({
    "chat_history": [
        HumanMessage(content="Hi, My name is Anas"),
        AIMessage(content="Hi, Anas. Its nice to meet you."),
        HumanMessage(content="Do you remeber my name ?")
    ]
})

AIMessage(content='Yes, Anas, I remember your name. 😊 \n\nHow can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 60, 'total_tokens': 82, 'completion_time': 0.04, 'prompt_time': 0.003364599, 'queue_time': 0.26687634, 'total_time': 0.043364599}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--71b3e633-fe5b-4873-9b05-f00b0d962525-0', usage_metadata={'input_tokens': 60, 'output_tokens': 22, 'total_tokens': 82})

In [22]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

In [23]:
config = {"configurable": {"session_id": "chat3"}}

In [25]:
with_message_history.invoke(
    [HumanMessage(content="I love to play cricket.")],
    config=config
)

AIMessage(content="That's great! Cricket is a wonderful sport. 🏏\n\nI'm happy to answer any questions you have about cricket. \n\nDo you have a specific question about the game, or would you like to chat about your favorite team, player, or match?  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 33, 'total_tokens': 95, 'completion_time': 0.112727273, 'prompt_time': 0.003100242, 'queue_time': 0.258470917, 'total_time': 0.115827515}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--445a9b19-fb34-4bf3-ad5a-7bc54e5e0db0-0', usage_metadata={'input_tokens': 33, 'output_tokens': 62, 'total_tokens': 95})

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="Do you remeber what I love to play ?")],
    config=config
)
response.content

'Yes, you told me you love to play cricket! 🏏 \n\nIs there anything specific you want to talk about related to cricket?  Perhaps your favorite team, a recent match you watched, or a particular player you admire?  😊  \n\n\n\n\n'

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all of the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="chat_history")
    ]
)
chain = prompt | model

In [29]:
response = chain.invoke({
    "chat_history": [HumanMessage(content="Hi, My name is Anas")],
    "language": "Hindi"
})
response.content

'नमस्ते अनस! \n\nमुझे बहुत खुशी है आपसे मिलने में! \n\nआप क्या पूछना चाहते हैं? मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूंगा। \n'

Now Lets wrap thsi more complicated chain in a Message History Class. This time  as there are multiple keys in the input, we need to specify the correct key to save the chat history

In [30]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all of the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="chat_history")
    ]
)
# chain
chain = prompt | model

In [31]:
# Store
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [32]:
# Define the runnable
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="chat_history"
)

In [33]:
config = {"configurable": {"session_id": "chat4"}}

In [34]:
response = with_message_history.invoke({
    "chat_history": [HumanMessage(content="Hi, My name is Anas Malik.")],
    "language": "Hindi"
}, config=config)
response.content

'नमस्ते अनास मलिक!  मैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआपका कोई सवाल है जो मैं आपके लिए उत्तर दे सकता हूँ? 😊 \n'

In [35]:
response = with_message_history.invoke({
    "chat_history": [HumanMessage(content="What is my name ?")],
    "language": "Hindi"
}, config=config)
response.content

'आपका नाम अनास मलिक है।  😊  \n'

## Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages': Helper to reduce how many messages we are sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [36]:
from langchain_core.messages import SystemMessage, trim_messages

In [47]:
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [48]:
messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi, I am bob"),
    AIMessage(content="Hi"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="Whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No Problem."),
    HumanMessage(content="Have fun ?"),
    AIMessage(content="yes!")
]

In [49]:
trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No Problem.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Have fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [52]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [58]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all of the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [59]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
    | StrOutputParser()
)

In [60]:
chain.invoke({
    "messages": messages + [HumanMessage(content="What ice cream do I like ?")],
    "language": "French"
})

"Comme je suis un assistant virtuel, je n'ai pas accès à vos informations personnelles, comme vos préférences en matière de glace. \n\nPourrais-tu me dire quelle est ta glace préférée ?  🍦 😋 \n\n\n\n"

In [61]:
chain.invoke({
    "messages": messages + [HumanMessage(content="What math problem I asked ?")],
    "language": "Hindi"
})

'तुम्हेंने 2 + 2 का जवाब पूछा था।  \n'

Now wrap this in the Message History

In [62]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [65]:
config = {"configurable": {"session_id": "chat5"}}

In [67]:
with_message_history.invoke({
    "messages": HumanMessage(content="Hello, My name is Anas Malik and I like to play cricket"),
    "language": "Hindi"
}, config=config)

'नमस्ते अनास मलिक! 🏏 \n\nमुझे खुशी है कि आप मेरी मदद चाहते हैं।  क्रिकेट खेलना बहुत मजेदार होता है! आप किस टीम का समर्थन करते हैं? \n'

In [68]:
with_message_history.invoke({
    "messages": HumanMessage(content="I lives in India and I am an AI Engineer"),
    "language": "Hindi"
}, config=config)

'नमस्ते! मैं आपकी सहायता करने के लिए यहाँ हूँ। आप मुझे जो भी प्रश्न पूछेंगे, मैं अपनी पूरी क्षमता के अनुसार उत्तर देने की कोशिश करूँगा।  \n\nक्या आप कोई प्रश्न पूछना चाहेंगे? \n\n'

In [69]:
with_message_history.invoke({
    "messages": HumanMessage(content="What all things you know about me."),
    "language": "Hindi"
}, config=config)

'मुझे आपके बारे में कुछ भी नहीं पता। मैं एक बड़ा भाषा मॉडल हूँ, लेकिन मेरी याददाश्त सीमित है। प्रत्येक बातचीत हमारी शुरुआत से ताज़ा होती है। \n\nआप मुझे अपने बारे में बताना चाहें तो मुझे खुशी होगी! \n\n'

In [71]:
store["chat5"]

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, My name is Anas Malik and I like to play cricket', additional_kwargs={}, response_metadata={}), AIMessage(content='नमस्ते अनास मलिक! 🏏 \n\nमुझे खुशी है कि आप मेरी मदद चाहते हैं।  क्रिकेट खेलना बहुत मजेदार होता है! आप किस टीम का समर्थन करते हैं? \n', additional_kwargs={}, response_metadata={}), HumanMessage(content='I lives in India and I am an AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content='नमस्ते! मैं आपकी सहायता करने के लिए यहाँ हूँ। आप मुझे जो भी प्रश्न पूछेंगे, मैं अपनी पूरी क्षमता के अनुसार उत्तर देने की कोशिश करूँगा।  \n\nक्या आप कोई प्रश्न पूछना चाहेंगे? \n\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What all things you know about me.', additional_kwargs={}, response_metadata={}), AIMessage(content='मुझे आपके बारे में कुछ भी नहीं पता। मैं एक बड़ा भाषा मॉडल हूँ, लेकिन मेरी याददाश्त सीमित है। प्रत्येक बातचीत हमारी शुरुआत से ताज़ा होती है। \n\nआप मुझे अपने बारे म